# Query 2

This is an example query on the NEXUS database to obtain data from XNAT based on various metadata that have been provided using the Electronic Lab Notebook. 

- “manufacturer” EYELINK                        <-- Acquisition Object Header
- “Psychiatric History” contains depression     <-- Subject Level Form
- “Eye Dominance” Right                         <-- Subject Level Form
    
Query 2 output parameters required:
- project, subject, session, scans

Expected: nexus1, NA002, NA002_ET_1

In [1]:
import pandas as pd
import setup
import nexussdk as nexus
from SPARQLWrapper import SPARQLWrapper, JSON, POST, GET, POSTDIRECTLY, CSV
from questionnaire_query_helper import generate_sparql_compliant_variable_name, generate_question_hash, query_constructor
config = setup.config
import NexusSparqlQuery as qns

# get script arguments
nexus_deployment = config['nexus']['deployment']
org = config['nexus']['org']
project = config['nexus']['project']
token = config['nexus']['token_file']
f = open(token, 'r')
token = f.read().rstrip()

sparqlview_endpoint = setup.sparqlview_endpoint
sparqlview_wrapper = setup.sparqlview_wrapper

nexus = setup.nexus

print("Connected")


Connected


In [2]:
prefix = """
prefix nxv: <https://bluebrain.github.io/nexus/vocabulary/>
PREFIX fhir: <http://hl7.org/fhir/> 
PREFIX kcnietl: <http://www.camh.ca/kcnietl/> 
PREFIX nidm: <http://purl.org/nidash/nidm#> 
PREFIX prov: <http://www.w3.org/ns/prov#> 
PREFIX sio: <http://semanticscience.org/resource/> 
PREFIX dct: <http://purl.org/dc/terms/> 
PREFIX bids: <https://wolfborg.github.io/nidm/bids.html#>
"""

select_list = [
    '?study_id',
    '?subject_id',
    '?session_id',
    '?acquisition_id',
    '?device_manufacturer'
]

base_query = """
        ##### Get Study Information
        ?study_uri a fhir:ResearchStudy .
        ?study_uri fhir:ResearchStudy.identifier/fhir:Identifier.value/fhir:value ?study_id .
        
        #### Get Subject Information        
        ?subject_uri fhir:ResearchSubject.study/fhir:link ?study_uri .
        ?subject_uri fhir:ResearchSubject.identifier/fhir:Identifier.value/fhir:value ?subject_id .
        
        #### Get Session Information
        ?session a nidm:Session .
        ?session prov:wasAssociatedWith ?subject_uri .
        ?session dct:identifier ?session_id .
        
        #### Get Acquisition Information
        ?acquisition a nidm:Acquisition .
        ?acquisition dct:isPartOf ?session .
        ?acquisition dct:identifier ?acquisition_id .
        
        #### Get Acquisition Object Information
        ?acquisition_object a nidm:AcquisitionObject .
        ?acquisition_object prov:wasGeneratedBy ?acquisition .
        ?acquisition_object bids:Manufacturer ?device_manufacturer .
        FILTER (?device_manufacturer = 'EYELINK') .
        
"""

subject_question_list = ['Eye Dominance', 'Psychiatric History']
for question in subject_question_list:
    sparql_compliant_variable_name = generate_sparql_compliant_variable_name(question)
    select_list.append(sparql_compliant_variable_name)
    base_query = base_query + query_constructor('?subject_uri', question, sparql_compliant_variable_name)


In [3]:
query = f"""
{prefix}

select distinct
    {' '.join(select_list)}
where
    {{
        {base_query}
    }}
"""


In [4]:
print(query)



prefix nxv: <https://bluebrain.github.io/nexus/vocabulary/>
PREFIX fhir: <http://hl7.org/fhir/> 
PREFIX kcnietl: <http://www.camh.ca/kcnietl/> 
PREFIX nidm: <http://purl.org/nidash/nidm#> 
PREFIX prov: <http://www.w3.org/ns/prov#> 
PREFIX sio: <http://semanticscience.org/resource/> 
PREFIX dct: <http://purl.org/dc/terms/> 
PREFIX bids: <https://wolfborg.github.io/nidm/bids.html#>


select distinct
    ?study_id ?subject_id ?session_id ?acquisition_id ?device_manufacturer ?Eye___Dominance ?Psychiatric___History
where
    {
        
        ##### Get Study Information
        ?study_uri a fhir:ResearchStudy .
        ?study_uri fhir:ResearchStudy.identifier/fhir:Identifier.value/fhir:value ?study_id .
        
        #### Get Subject Information        
        ?subject_uri fhir:ResearchSubject.study/fhir:link ?study_uri .
        ?subject_uri fhir:ResearchSubject.identifier/fhir:Identifier.value/fhir:value ?subject_id .
        
        #### Get Session Information
        ?session a

In [5]:
df = qns.sparql2dataframe(qns.query_sparql(query, sparqlview_wrapper))

df.loc[
    (df['device_manufacturer'] == 'EYELINK')
    & (df['Eye___Dominance'] == 'right')
    & (df['Psychiatric___History'].str.find('depression')==0)
    ].drop_duplicates()

,study_id,subject_id,session_id,acquisition_id,device_manufacturer,Eye___Dominance,Psychiatric___History
0,nexus1,NA002,NA002_ET_1,DurR6,EYELINK,right,depression
1,nexus1,NA002,NA002_ET_1,DurR5,EYELINK,right,depression
2,nexus1,NA002,NA002_ET_1,DurR4,EYELINK,right,depression
3,nexus1,NA002,NA002_ET_1,DurR3,EYELINK,right,depression
4,nexus1,NA002,NA002_ET_1,DurR2,EYELINK,right,depression
5,nexus1,NA002,NA002_ET_1,DurR1,EYELINK,right,depression
6,nexus1,NA001,NA001_ET_1,Dur1,EYELINK,right,depression
7,nexus2,NB002,NB002_ET_1,DurR8,EYELINK,right,depression
8,nexus2,NB002,NB002_ET_1,DurR7,EYELINK,right,depression
9,nexus2,NB002,NB002_ET_1,DurR6,EYELINK,right,depression


End.